In [1]:
import os
import json
import csv
import logging
import concurrent.futures
from typing import List, Dict, Tuple, Union
import pandas as pd

In [ ]:
# Configurer les logs
logging.basicConfig(filename='notebooks/log.txt', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [4]:
# Charger la configuration depuis un fichier JSON
with open('notebooks/config.json', 'r') as f:
    config = json.load(f)

PATH_TO_CSVS = config['path_to_csvs']
ENCODINGS = config['encodings']
PREFIXES = config['prefixes']
YEARS = config['years']

In [5]:
# Classe de chargement
class DataLoader:
    def __init__(self, base_path: str = PATH_TO_CSVS):
        self.base_path = base_path

    def get_delimiter(self, file_path: str, bytes: int = 4096) -> Union[str, None]:
        """
        Detecte automatiquement le separateur d'un fichier CSV.
        """
        try:
            with open(file_path, 'r') as file:
                data = file.read(bytes)
            sniffer = csv.Sniffer()
            delimiter = sniffer.sniff(data).delimiter
            return delimiter
        except Exception as e:
            logging.error(f"Erreur lors de la détection du délimiteur: {e}")
            return None

    def read_csv_file(self, file_path: str) -> Tuple[Union[pd.DataFrame, None], bool, Union[str, None]]:
        """
        Lecture d'un fichier CSV en essayant différents encodages.
        """
        if not os.path.exists(file_path):
            return None, False, f"Fichier non trouvé: {file_path}"
        
        delimiter = self.get_delimiter(file_path)
        if not delimiter:
            return None, False, f"Impossible de détecter le délimiteur pour le fichier: {file_path}"
        
        for encoding in ENCODINGS:
            try:
                df = pd.read_csv(file_path, low_memory=False, encoding=encoding, delimiter=delimiter)
                return df, True, None
            except (UnicodeDecodeError, pd.errors.ParserError) as e:
                logging.warning(f"Erreur avec l'encodage {encoding} pour le fichier {file_path}: {e}")
        
        return None, False, f"Impossible de lire le fichier {file_path} avec les encodages: {ENCODINGS}."

    def load_dataset(self, prefix: str, year: int) -> Tuple[str, Union[pd.DataFrame, None]]:
        """
        Charge un dataset pour un préfixe et une année donnée.
        """
        connector = '_' if year <= 2016 else '-'
        file_name = os.path.join(self.base_path, f'{prefix}{connector}{year}.csv')
        df, success, error = self.read_csv_file(file_name)
        if success:
            logging.info(f"Chargement réussi du fichier: {file_name}")
            return file_name, df
        else:
            logging.error(f"Erreur lors du chargement du fichier {file_name}: {error}")
            return file_name, None

    def load_datasets_parallel(self, prefixes: List[str], years: List[int]) -> Dict[str, List[pd.DataFrame]]:
        """
        Charge les datasets en parallèle pour les préfixes et années donnés.
        """
        datasets = {prefix: [] for prefix in prefixes}
        
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future_to_dataset = {
                executor.submit(self.load_dataset, prefix, year): (prefix, year)
                for prefix in prefixes for year in years
            }
            
            for future in concurrent.futures.as_completed(future_to_dataset):
                prefix, year = future_to_dataset[future]
                try:
                    file_name, df = future.result()
                    if df is not None:
                        datasets[prefix].append(df)
                except Exception as e:
                    logging.error(f"Erreur lors du chargement du dataset pour {prefix} {year}: {e}")
        
        return datasets

In [6]:
data_loader = DataLoader()
datasets = data_loader.load_datasets_parallel(PREFIXES, YEARS)

# Log du nombre de datasets chargés pour chaque préfixe
for prefix, dfs in datasets.items():
    logging.info(f"{prefix}: {len(dfs)} datasets chargés.")